In [ ]:
# 📌 Etapa 1: Instalar bibliotecas (caso necessário)
!pip install google-cloud-bigquery pandas --quiet

In [1]:
# 📌 Etapa 3: Importar bibliotecas
import pandas as pd
from google.cloud import bigquery

In [5]:
# 📌 Etapa 2: Autenticar com Google
from google.colab import auth
auth.authenticate_user()

In [6]:
# 📌 Etapa 4: Configurar cliente do BigQuery
project_id = 'meu-projeto-dbt'  # substitua com seu ID do projeto
dataset = 'meu_dataset'

client = bigquery.Client(project=project_id)

In [7]:
# 📌 Etapa 5: Definir nomes das tabelas
tabelas_gold = {
    'investment': f"{project_id}.{dataset}.gold_investment",
    'kpi': f"{project_id}.{dataset}.gold_kpi",
    'auxiliar': f"{project_id}.{dataset}.gold_auxiliar"
}


In [8]:
# 📌 Etapa 6: Visualizar o head de cada tabela
for nome, tabela in tabelas_gold.items():
    print(f"\n🔍 Head da tabela: {nome}")
    df = client.query(f"SELECT * FROM `{tabela}` LIMIT 5").to_dataframe()
    display(df)



🔍 Head da tabela: investment


,date,tv,radio,ooh,meta,google,tiktok,display_video
0,2023-01-26,5847.22,954.10,615.34,3769.66,5633.30,3635.53,1788.38
1,2023-02-07,6856.33,1276.32,1981.44,2822.47,8726.19,5904.19,2316.47
2,2023-05-13,5570.36,750.27,0.00,2704.28,6703.93,5212.50,2034.38
3,2023-07-19,5965.63,1029.58,0.00,4033.08,7456.49,5124.40,1829.89
4,2024-02-08,9019.14,1509.54,1342.55,4384.67,10457.13,7683.67,2540.45



🔍 Head da tabela: kpi


,date,sales
0,2023-01-23,7782332.54
1,2023-05-03,9372060.61
2,2024-12-20,81598600.42
3,2023-10-15,7729249.55
4,2024-05-27,77128283.19



🔍 Head da tabela: auxiliar


,date,holiday
0,2023-01-20,0.0
1,2023-07-07,0.0
2,2023-08-11,0.0
3,2023-08-18,0.0
4,2023-09-06,0.0


In [9]:
# 📌 Etapa 7: Validar se não há valores nulos em sales
df_kpi = client.query(f"SELECT * FROM `{tabelas_gold['kpi']}`").to_dataframe()
assert df_kpi['sales'].isnull().sum() == 0, "❌ Ainda há valores nulos em `sales`!"
print("✅ Coluna `sales` está totalmente preenchida.")

✅ Coluna `sales` está totalmente preenchida.


In [10]:
# 📌 Etapa 8: Validar se todas as colunas de investimento estão preenchidas
df_inv = client.query(f"SELECT * FROM `{tabelas_gold['investment']}`").to_dataframe()
cols_invest = ['tv', 'radio', 'ooh', 'meta', 'google', 'tiktok', 'display_video']
assert df_inv[cols_invest].isnull().sum().sum() == 0, "❌ Ainda há valores nulos em investimento!"
print("✅ Todas as colunas de investimento estão preenchidas.")

✅ Todas as colunas de investimento estão preenchidas.


In [11]:
# 📌 Etapa 9: Validar se holidays estão preenchidos (ou vazios, mas não nulos)
df_aux = client.query(f"SELECT * FROM `{tabelas_gold['auxiliar']}`").to_dataframe()
assert df_aux['holiday'].isnull().sum() == 0, "❌ Ainda há valores nulos em `holiday`!"
print("✅ Coluna `holiday` está sem nulos.")

✅ Coluna `holiday` está sem nulos.


In [12]:
# 📌 Teste 1: Verificar se todas as vendas são > 0
sales_invalidas = df_kpi[df_kpi['sales'] <= 0]
assert sales_invalidas.empty, f"❌ Existem {len(sales_invalidas)} linhas com sales <= 0"
print("✅ Todas as vendas (`sales`) são maiores que 0.")

✅ Todas as vendas (`sales`) são maiores que 0.


In [17]:
# 📌 Teste 2: Verificar se colunas de investimento têm todos os valores > 0
for col in cols_invest:
    invalidos = df_inv[df_inv[col] < 0]
    assert invalidos.empty, f"❌ Existem valores <= 0 na coluna {col}"
print("✅ Todas as colunas de investimento possuem apenas valores >= 0.")

✅ Todas as colunas de investimento possuem apenas valores >= 0.


In [21]:
# 📌 Teste 3: Validar se `holiday` é binário (0 ou 1)
valores_unicos = df_aux['holiday'].dropna().astype(int).unique()
valores_validos = set([0, 1, '0', '1', True, False, 'True', 'False'])

if all(str(v) in ['0', '1'] for v in valores_unicos):
    print("✅ Coluna `holiday` contém apenas valores binários (0 e 1).")
else:
    raise ValueError(f"❌ A coluna `holiday` contém valores não binários: {valores_unicos}")

✅ Coluna `holiday` contém apenas valores binários (0 e 1).


In [31]:
# 📌 Etapa 10: Juntar as três tabelas em um único dataframe
query = f"""
SELECT
    inv.date,
    inv.tv, inv.radio, inv.ooh, inv.meta, inv.google, inv.tiktok, inv.display_video,
    kpi.sales,
    aux.holiday
FROM `{tabelas_gold['investment']}` inv
LEFT JOIN `{tabelas_gold['kpi']}` kpi ON inv.date = kpi.date
LEFT JOIN `{tabelas_gold['auxiliar']}` aux ON inv.date = aux.date
"""

df_joined = client.query(query).to_dataframe()

# Converter 'date' para datetime
df_joined['date'] = pd.to_datetime(df_joined['date'], errors='coerce')

# Converter 'holiday' para inteiro (com tratamento de nulos, se necessário)
df_joined['holiday'] = df_joined['holiday'].fillna(0).astype(int)
print("✅ Tabela final com join das três gold criada!")
df_joined.sort_values(by='date').head()


✅ Tabela final com join das três gold criada!


,date,tv,radio,ooh,meta,google,tiktok,display_video,sales,holiday
200,2023-01-01,6588.79,1162.52,990.38,2977.19,8932.64,4556.60,2315.72,35768254.89,1
0,2023-01-02,7834.65,1513.92,1639.37,3026.25,8681.33,3851.36,1966.45,10222134.66,0
214,2023-01-03,7444.58,786.07,1800.90,4366.05,7732.49,3646.35,1977.41,9791019.64,0
562,2023-01-04,6731.67,1431.54,0.00,4129.13,7624.79,3143.86,2372.24,9989567.91,0
265,2023-01-05,5470.03,0.00,0.00,4638.42,6035.70,5192.94,1765.80,6910931.76,0
